In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

warnings.filterwarnings('ignore')

In [2]:
try:
    import google.colab
    from google.colab import drive

    !uv pip install anomalib
    !uv pip install open-clip-torch
    !uv pip install qwen-vl-utils
    !uv pip install transformers==4.52.4
    !uv pip install langchain-chroma langchain-huggingface
    !uv pip install langchain_community pypdf beautifulsoup4
    !uv pip install -U bitsandbytes
    !uv pip install -U torchao

    drive.mount('/content/drive', force_remount=True)

    # Colab Root
    PROJECT_ROOT = Path('/content/drive/Othercomputers/Mac/multiModal_anomaly_report') # 본인 경로 수정: Mac/Window
    DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"

except ImportError:

    # Local Root
    PROJECT_ROOT = Path.cwd().parents[1]
    DATA_ROOT = PROJECT_ROOT / "datasets" / "MMAD"

os.chdir(PROJECT_ROOT) # 현재 경로 수정
print(f"Current working directory: {os.getcwd()}")

Using Python 3.12.12 environment at: /usr
Resolved 98 packages in 1.45s
Prepared 12 packages in 486ms
Installed 12 packages in 18ms
 + anomalib==2.2.0
 + freia==0.2
 + imagecodecs==2026.1.14
 + jsonargparse==4.46.0
 + kornia==0.8.2
 + kornia-rs==0.1.10
 + lightning==2.6.1
 + lightning-utilities==0.15.2
 + pytorch-lightning==2.6.1
 + rich-argparse==1.7.2
 + torchmetrics==1.8.2
 + typeshed-client==2.8.2
Using Python 3.12.12 environment at: /usr
Resolved 57 packages in 229ms
Prepared 2 packages in 97ms
Installed 2 packages in 5ms
 + ftfy==6.3.1
 + open-clip-torch==3.2.0
Using Python 3.12.12 environment at: /usr
Resolved 9 packages in 164ms
Prepared 2 packages in 539ms
Installed 2 packages in 7ms
 + av==16.1.0
 + qwen-vl-utils==0.0.14
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 174ms
Prepared 3 packages in 450ms
Uninstalled 3 packages in 356ms
Installed 3 packages in 43ms
 - huggingface-hub==1.4.1
 + huggingface-hub==0.36.2
 - tokenizers==0.22.2
 + tokenizers==0.21.4


In [3]:
# RAG Dataet path
DOMAIN_KNOWLEDGE_ROOT = DATA_ROOT / "domain_knowledge.json"
PDF_PATH = DATA_ROOT / "packaging_guide.pdf"
CFIA_JSON_PATH = DATA_ROOT / "cfia_knowledge.json"

In [4]:
from src.rag import Indexer, Retrievers, PDFKnowledgeLoader, JSONKnowledgeLoader

pdf_loader = PDFKnowledgeLoader(pdf_path=PDF_PATH, chunk_size=1000, chunk_overlap=100)
pdf_docs = pdf_loader.load()
print(f"PDF Total: {len(pdf_docs)} chunks")

cfia_docs = JSONKnowledgeLoader(CFIA_JSON_PATH).load()
print(f"CFIA Total: {len(cfia_docs)} docs")

PDF Total: 20 chunks
CFIA Total: 26 docs


In [5]:
from src.rag import RAGEvaluator, TEST_QUERIES_MMAD, TEST_QUERIES_EXTERNAL
import shutil

# Config A: JSON only
indexers = Indexer(
    json_path=DOMAIN_KNOWLEDGE_ROOT,
    persist_dir="vectorstore/domain_knowledge"
)
vs_config = indexers.get_or_create()
domain_docs = indexers.load_documents()

# Config B: JSON + PDF
shutil.rmtree("vectorstore/domain_knowledge_pdf", ignore_errors=True)
vs_config_pdf = Indexer(
    persist_dir="vectorstore/domain_knowledge_pdf"
).build_index_from(domain_docs + pdf_docs)

# Config C: JSON + CFIA
shutil.rmtree("vectorstore/domain_knowledge_cfia", ignore_errors=True)
vs_config_cfia = Indexer(
    persist_dir="vectorstore/domain_knowledge_cfia"
).build_index_from(domain_docs + cfia_docs)

# Config D: JSON + PDF + CFIA
shutil.rmtree("vectorstore/domain_knowledge_pdf_cfia", ignore_errors=True)
vs_config_pdf_cfia = Indexer(
    persist_dir="vectorstore/domain_knowledge_pdf_cfia"
).build_index_from(domain_docs + pdf_docs + cfia_docs)

queries = TEST_QUERIES_MMAD + TEST_QUERIES_EXTERNAL

result_config = RAGEvaluator(Retrievers(vs_config)).evaluate(queries, k=3)
result_config_pdf = RAGEvaluator(Retrievers(vs_config_pdf)).evaluate(queries, k=3)
result_config_cfia = RAGEvaluator(Retrievers(vs_config_cfia)).evaluate(queries, k=3)
result_config_pdf_cfia = RAGEvaluator(Retrievers(vs_config_pdf_cfia)).evaluate(queries, k=3)

print(f"{'Config':<12} {'Hit Rate':>10} {'MRR':>10}")
print(f"{'A (JSON)':12} {result_config['hit_rate']:>10.3f} {result_config['mrr']:>10.3f}")
print(f"{'B (+PDF)':12} {result_config_pdf['hit_rate']:>10.3f} {result_config_pdf['mrr']:>10.3f}")
print(f"{'C (+CFIA)':12} {result_config_cfia['hit_rate']:>10.3f} {result_config_cfia['mrr']:>10.3f}")
print(f"{'D (+PDF + CFIA)':12} {result_config_pdf_cfia['hit_rate']:>10.3f} {result_config_pdf_cfia['mrr']:>10.3f}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Config         Hit Rate        MRR
A (JSON)          0.429      0.429
B (+PDF)          0.786      0.655
C (+CFIA)         0.643      0.607
D (+PDF + CFIA)      0.929      0.798


In [21]:
from src.utils.loaders import load_json

# setting
# gemma3-4b-int4, gemma3-12b-int4, gemma3-27b-int4
# gemma3-4b-int8, gemma3-12b-int8, gemma3-27b-int8

LLM = "gemma3-12b"
MMAD_CLASS_JSON = DATA_ROOT / "mmad_10classes.json"
OUTPUT_DIR = f"output/{LLM}"
OUTPUT_RAG_ROOT = Path(OUTPUT_DIR) / "rag"

OUTPUT_AD = OUTPUT_RAG_ROOT / "AD"
OUTPUT_ORIGIN = OUTPUT_RAG_ROOT / "original"
OUTPUT_PDF = OUTPUT_RAG_ROOT / "pdf"
OUTPUT_CFIA = OUTPUT_RAG_ROOT / "cfia"
OUTPUT_PDF_CFIA = OUTPUT_RAG_ROOT / "pdf_cfia"
SAMPLE_PER_FOLDER = 3  # 빠른 테스트: 폴더당 3장

In [22]:
class_10_json = load_json(MMAD_CLASS_JSON)

# With RAG
!python scripts/run_experiment.py \
    --llm {LLM} \
    --ad-model "patchcore" \
    --rag \
    --data-root {DATA_ROOT} \
    --sample-per-folder {SAMPLE_PER_FOLDER} \
    --output-dir {OUTPUT_ORIGIN} \
    --mmad-json {MMAD_CLASS_JSON} \
    --batch-mode true

Stratified sampling: 3장/폴더, 33폴더
  Total: 4224 -> Sampled: 99 (normal=30, anomaly=69)
2026-02-22 14:41:38.341517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771771298.365099   45212 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771771298.372990   45212 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771771298.393678   45212 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771771298.393718   45212 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W

In [17]:
# json + pdf
!python scripts/run_experiment.py \
    --llm {LLM} --ad-model "patchcore" --rag \
    --rag-persist-dir vectorstore/domain_knowledge_pdf \
    --data-root {DATA_ROOT} --mmad-json {MMAD_CLASS_JSON} \
    --sample-per-folder {SAMPLE_PER_FOLDER} \
    --output-dir {OUTPUT_PDF} --batch-mode true

Stratified sampling: 3장/폴더, 33폴더
  Total: 4224 -> Sampled: 99 (normal=30, anomaly=69)
2026-02-22 13:33:45.519829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771767225.544580   26383 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771767225.552651   26383 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771767225.574061   26383 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771767225.574104   26383 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W

In [18]:
from src.utils.loaders import load_json

# setting
# gemma3-4b-int4, gemma3-12b-int4, gemma3-27b-int4
# gemma3-4b-int8, gemma3-12b-int8, gemma3-27b-int8

LLM = "gemma3-12b-int4"

In [19]:
# With RAG
# !uv pip install --upgrade torchao
!python scripts/run_experiment.py \
    --llm {LLM} \
    --ad-model "patchcore" \
    --rag \
    --data-root {DATA_ROOT} \
    --sample-per-folder {SAMPLE_PER_FOLDER} \
    --output-dir {OUTPUT_ORIGIN} \
    --mmad-json {MMAD_CLASS_JSON} \
    --batch-mode true

Stratified sampling: 3장/폴더, 33폴더
  Total: 4224 -> Sampled: 99 (normal=30, anomaly=69)
2026-02-22 13:39:26.526578: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771767566.549894   28129 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771767566.557590   28129 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771767566.577145   28129 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771767566.577180   28129 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W

In [20]:
# json + pdf
!python scripts/run_experiment.py \
    --llm {LLM} --ad-model "patchcore" --rag \
    --rag-persist-dir vectorstore/domain_knowledge_pdf \
    --data-root {DATA_ROOT} --mmad-json {MMAD_CLASS_JSON} \
    --sample-per-folder {SAMPLE_PER_FOLDER} \
    --output-dir {OUTPUT_PDF} --batch-mode true

Stratified sampling: 3장/폴더, 33폴더
  Total: 4224 -> Sampled: 99 (normal=30, anomaly=69)
2026-02-22 13:48:11.571967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771768091.594859   30715 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771768091.602519   30715 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771768091.622021   30715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771768091.622055   30715 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W

In [14]:
# With RAG
LLM = "gemma3-27b-int4"
!python scripts/run_experiment.py \
    --llm {LLM} \
    --ad-model "patchcore" \
    --rag \
    --data-root {DATA_ROOT} \
    --sample-per-folder {SAMPLE_PER_FOLDER} \
    --output-dir {OUTPUT_ORIGIN} \
    --mmad-json {MMAD_CLASS_JSON} \
    --batch-mode true

Stratified sampling: 3장/폴더, 33폴더
  Total: 4224 -> Sampled: 99 (normal=30, anomaly=69)
2026-02-22 13:07:53.776582: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771765673.801264   18935 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771765673.809239   18935 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771765673.830281   18935 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771765673.830324   18935 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W

In [12]:
# json + pdf
!python scripts/run_experiment.py \
    --llm {LLM} --ad-model "patchcore" --rag \
    --rag-persist-dir vectorstore/domain_knowledge_pdf \
    --data-root {DATA_ROOT} --mmad-json {MMAD_CLASS_JSON} \
    --sample-per-folder {SAMPLE_PER_FOLDER} \
    --output-dir {OUTPUT_PDF} --batch-mode true

Stratified sampling: 3장/폴더, 33폴더
  Total: 4224 -> Sampled: 99 (normal=30, anomaly=69)
2026-02-22 12:53:52.673746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771764832.696405   14870 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771764832.703855   14870 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771764832.722870   14870 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771764832.722903   14870 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W

In [13]:
# 결과 비교
configs = {
    # "Baseline": OUTPUT_DIR,
    # "AD only": OUTPUT_AD,
    "AD+RAG": OUTPUT_ORIGIN,
    "AD+RAG (+PDF)": OUTPUT_PDF,
}

def load_latest_meta(output_dir):
    files = sorted(Path(output_dir).glob("*.meta.json"))
    if not files:
        return None
    return json.load(open(files[-1]))

rows = {}
for label, out_dir in configs.items():
    meta = load_latest_meta(out_dir)
    rows[label] = meta

# 기준: Baseline accuracy
# bl_acc = rows["Baseline"]["accuracy"] if rows["Baseline"] else 0

print(f"{'Config':<12} {'Accuracy':>10} {'Correct':>9} {'Total':>7} {'Diff':>8}")
print("=" * 52)
for label, meta in rows.items():
    if meta is None:
        print(f"{label:<12} {'결과없음':>10}")
        continue
    acc  = meta.get("accuracy", 0)
    cor  = meta.get("total_correct", 0)
    tot  = meta.get("total_questions", 0)
    # diff = acc - bl_acc
    # sign = "+" if diff > 0 else ""
    # diff_str = f"{sign}{diff:.2f}" if label != "Baseline" else "-"
    print(f"{label:<12} {acc:>9.2f}% {cor:>9} {tot:>7}") # {diff_str:>8}

def load_latest_answers(output_dir):
    files = [f for f in sorted(Path(output_dir).glob("answers_*.json")) if ".meta." not in f.name]
    return json.load(open(files[-1])) if files else []

answers = {label: load_latest_answers(d) for label, d in configs.items()}

# good vs anomaly 정확도
print(f"{'Config':<25} {'good':>8} {'anomaly':>10}")
print("-" * 46)
for label, ans in answers.items():
    good    = [a for a in ans if "/good/" in a["image"]]
    anomaly = [a for a in ans if "/good/" not in a["image"]]
    g_acc = sum(a["gpt_answer"]==a["correct_answer"] for a in good) / len(good) * 100 if good else 0
    a_acc = sum(a["gpt_answer"]==a["correct_answer"] for a in anomaly) / len(anomaly) * 100 if anomaly else 0
    print(f"{label:<25} {g_acc:>7.1f}% {a_acc:>9.1f}%")

Config         Accuracy   Correct   Total     Diff
AD+RAG           81.56%       367     450
AD+RAG (+PDF)     81.56%       367     450
Config                        good    anomaly
----------------------------------------------
AD+RAG                       74.3%      84.7%
AD+RAG (+PDF)                74.3%      84.7%
